Ref: 

http://gaussianprocess.org/gpml/data/

Split ratio: https://github.com/Kaixhin/SARCOS

In [1]:
import sys
import tensorflow as tf
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

sys.path.append("../")

In [2]:
print(tf.__version__)

2.14.1


In [3]:
filepath_train = 'sarcos_inv.csv'
filepath_test = 'sarcos_inv_test.csv'

features = [c for c in range(0,21)]
targets = [c for c in range(21,28)]

# read training set
df_train = pd.read_csv(filepath_train)
#df_train = df_train.iloc[:,columns]

# read test set
df_test = pd.read_csv(filepath_test)
#df_test = df_test.iloc[:,columns]
df_train.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27'],
      dtype='object')

In [4]:
len(targets)

7

In [5]:
from sklearn.model_selection import train_test_split

y_train = df_train.iloc[:,targets].values
X_train = df_train.iloc[:,features].values
y_test = df_test.iloc[:,targets].values
X_test = df_test.iloc[:,features].values

X_train, X_vald, y_train, y_vald = train_test_split(X_train, y_train, test_size=4500, random_state=0)

print(f'X_train.shape: {X_train.shape}')
print(f'X_vald.shape: {X_vald.shape}')
print(f'X_test.shape: {X_test.shape}')
print(f'y_train.shape: {y_train.shape}')
print(f'y_vald.shape: {y_vald.shape}')
print(f'y_test.shape: {y_test.shape}')

X_train.shape: (39984, 21)
X_vald.shape: (4500, 21)
X_test.shape: (4449, 21)
y_train.shape: (39984, 7)
y_vald.shape: (4500, 7)
y_test.shape: (4449, 7)


In [6]:
# Normalize target column
'''
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
y_train = scaler.fit_transform(y_train)
y_vald = scaler.transform(y_vald)
y_test = scaler.transform(y_test)

print(y_train.shape)
'''

(39984, 7)


In [6]:
def array_to_dataset(data, target, shuffle=True, batch_size=128):
    ds = tf.data.Dataset.from_tensor_slices((data, target))
    if shuffle:
        ds = ds.shuffle(batch_size*2).batch(batch_size).prefetch(batch_size)
    else:
        ds = ds.batch(batch_size)
    return ds

batch_size = 2048
train_ds = array_to_dataset(X_train, y_train, batch_size=batch_size)
vald_ds = array_to_dataset(X_vald, y_vald, shuffle=False, batch_size=batch_size)
test_ds = array_to_dataset(X_test, y_test, shuffle=False, batch_size=batch_size)

In [8]:
from ife import IFENetRegressor

n_features = X_train.shape[1]
_, counts = np.unique(y_train, return_counts=True)
n_response = len(targets)
ife_num_layers = 1
clf_num_layers = 1
clf_hidden_units = [64]
reduction_layer = 'average'
num_att = 64
r = 4.15

print(f'n_response: {n_response}')
print(f'n_features: {n_features}')

ife_params = {'n_features': n_features,
              'n_outputs': n_response,
              'num_att': num_att,
              'r': r,
              'ife_num_layers': ife_num_layers, 
              'clf_num_layers': clf_num_layers,
              'clf_hidden_units': clf_hidden_units,
              'reduction_layer': reduction_layer
             }
model = IFENetRegressor(**ife_params)

n_response: 7
n_features: 21


In [9]:
x = tf.keras.Input(shape=(n_features,))
# x = tf.keras.layers.Input(shape=input_shape)
model = tf.keras.models.Model(inputs=[x], outputs=model.call(x))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 21)]                 0         []                            
                                                                                                  
 tf.compat.v1.shape (TFOpLa  (2,)                         0         ['input_1[0][0]']             
 mbda)                                                                                            
                                                                                                  
 preprocess_batch_norm (Bat  (None, 21)                   84        ['input_1[0][0]']             
 chNormalization)                                                                                 
                                                                                              

In [30]:
loss_fn = tf.keras.losses.MeanSquaredError()

lr = 0.015
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=lr, 
                                                              decay_steps=2000,
                                                              decay_rate=0.95,
                                                              staircase=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

checkpoint_path = 'checkpoints/ifeNet_sarcos.h5'
patience = 100
callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience, monitor='val_loss'),
             tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, monitor='val_loss')]

epochs = 10
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['mse'])

In [ ]:
saved_model_path = 'saved_model/ifeNet_sarcos.h5'
model.fit(train_ds, validation_data=vald_ds, epochs=epochs, callbacks=callbacks, verbose=2)
model.load_weights(checkpoint_path)
model.save_weights(saved_model_path)

In [9]:
from ife import IFENetRegressor

n_features = X_train.shape[1]
_, counts = np.unique(y_train, return_counts=True)
n_response = len(targets)
ife_num_layers = 1
clf_num_layers = 1
clf_hidden_units = [128]
reduction_layer = 'flatten'
num_att = 128
r = 4.0

print(f'n_response: {n_response}')
print(f'n_features: {n_features}')

ife_params = {'n_features': n_features,
              'n_outputs': n_response,
              'num_att': num_att,
              'r': r,
              'ife_num_layers': ife_num_layers, 
              'clf_num_layers': clf_num_layers,
              'clf_hidden_units': clf_hidden_units,
              'reduction_layer': reduction_layer
             }
model = IFENetRegressor(**ife_params)
model.build(input_shape=(None,n_features))

path_saved_model = 'saved_model/ifeNet_sarcos_att_128.h5'
model.load_weights(path_saved_model)

n_response: 7
n_features: 21


In [10]:
y_pred = np.empty((0,n_response))
y_test = np.empty((0,n_response))

for data,label in test_ds:
    y_hat = model(data)
    y_pred = np.append(y_pred, y_hat, axis=0)
    #print(y_hat.shape)
    
    label = label.numpy()
    y_test = np.append(y_test, label, axis=0)
    #print(label.shape)


In [12]:
print(f'R2 Score: {r2_score(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

R2 Score: 0.9906084330078384
MSE: 0.3850258116224076
MAE: 0.3828897627707713


In [14]:
feat_scores = model.input_scores
feat_scores = np.mean(feat_scores, axis=(0,1))

feat_rank = {}
for col,score in zip(features,feat_scores):
    #print(f'{col}: {score}')
    feat_rank[col] = score

df_feat_rank = pd.DataFrame(list(feat_rank.items()), columns=['Feature', 'Score'])
df_feat_rank.sort_values(by='Score', ascending=False)

,Feature,Score
14,14,0.192080
0,0,0.096904
17,17,0.077443
1,1,0.071178
2,2,0.063495
15,15,0.063181
7,7,0.049514
16,16,0.048306
3,3,0.038565
12,12,0.032125
